In [2]:
pip install opencv-python

     --------------------------------------- 38.2/38.2 MB 13.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install keras

Note: you may need to restart the kernel to use updated packages.
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)


In [3]:
pip install tensorflow

In [1]:
import numpy as np
import argparse
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
import os

In [6]:
train_data = 'E://Project/Emotion Detection/data/train'
validate_data = 'E://Project/Emotion Detection/data/test'

In [7]:
train_data_gen = ImageDataGenerator (
    rescale=1./255,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip= True,
    fill_mode='nearest'
)

In [8]:
validate_data_gen= ImageDataGenerator(rescale=1./255)

In [9]:
train_generate= train_data_gen.flow_from_directory(
    train_data,
    color_mode="grayscale",
    target_size=(48,48),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

Found 28709 images belonging to 7 classes.


In [10]:
validate_generate= validate_data_gen.flow_from_directory(
    validate_data,
    color_mode="grayscale",
    target_size=(48,48),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

Found 7178 images belonging to 7 classes.


In [11]:
labels = ['angry', 'disgust', 'fear', 'happy','neutral', 'sad', 'surprise']

In [19]:
image, label= train_generate.__next__()

In [20]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape = (48, 48, 1)))

cnn_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D(pool_size=(2, 2), strides=(4, 4)))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D(pool_size=(2, 2), strides=(4, 4)))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2), strides=(4, 4), padding='same'))
cnn_model.add(Dropout(0.2))


cnn_model.add(Flatten())
cnn_model.add(Dense(512, activation='relu'))
cnn_model.add(Dropout(0.2))

cnn_model.add(Dense(7, activation='softmax'))

cnn_model.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])
print(cnn_model.summary)

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x00000220661AD030>>


In [14]:
train_path = 'E://Project/Emotion Detection/data/train'
test_path = 'E://Project/Emotion Detection/data/test'

num_train_imgs = 0
for root, dir, files in os.walk(train_path):
    num_train_imgs += len(files)

num_test_imgs = 0
for root, dir, files in os.walk(test_path):
    num_test_imgs += len(files)

epochs=30
cnn_history = cnn_model.fit(train_generate, 
                            steps_per_epoch=num_train_imgs//32 , 
                            epochs=epochs, 
                            validation_data=validate_generate, 
                            validation_steps=num_test_imgs//32,
                            verbose=1, 
                            shuffle=True)

Epoch 1/30
897/897 [==============================] - 805s 892ms/step - loss: 1.8132 - accuracy: 0.2505 - val_loss: 1.7374 - val_accuracy: 0.2866
Epoch 2/30
897/897 [==============================] - 279s 311ms/step - loss: 1.7459 - accuracy: 0.2885 - val_loss: 1.6530 - val_accuracy: 0.3467
Epoch 3/30
897/897 [==============================] - 410s 457ms/step - loss: 1.6986 - accuracy: 0.3182 - val_loss: 1.6030 - val_accuracy: 0.3655
Epoch 4/30
897/897 [==============================] - 137s 153ms/step - loss: 1.6531 - accuracy: 0.3440 - val_loss: 1.5234 - val_accuracy: 0.4132
Epoch 5/30
897/897 [==============================] - 146s 162ms/step - loss: 1.6111 - accuracy: 0.3625 - val_loss: 1.5032 - val_accuracy: 0.4146
Epoch 6/30
897/897 [==============================] - 140s 156ms/step - loss: 1.5889 - accuracy: 0.3785 - val_loss: 1.4984 - val_accuracy: 0.4167
Epoch 7/30
897/897 [==============================] - 140s 156ms/step - loss: 1.5694 - accuracy: 0.3883 - val_loss: 1.4730 -

In [28]:
cnn_model.save('cnn_model.h5')